In [1]:
import gen_dataset_810 as mod_810fr
# importer datetime pour typer les donnees de dates des chaines et des videos
import datetime
# module https://810.fr pour traitement des tags des videos
import gen_tags_visu_810 as tv810
# module https://810.fr pour casser les gros dataframe ou activites mensuelles
import gen_activitemensuelle_data_810 as amd810



# Définition des dates limites pour les filtres de https://s01.810.fr
liste_mois = [{"mois" : "Janvier", "dates" : {"datemin" : datetime.datetime(2022, 1, 1, 0, 0, 0), "datemax" : datetime.datetime(2022, 2, 1, 0, 0, 0)}},
                {"mois" : "Février", "dates" : {"datemin" : datetime.datetime(2022, 2, 1, 0, 0, 0), "datemax" : datetime.datetime(2022, 3, 1, 0, 0, 0)}},
                {"mois" : "Mars", "dates" : {"datemin" : datetime.datetime(2022, 3, 1, 0, 0, 0), "datemax" : datetime.datetime(2022, 4, 1, 0, 0, 0)}}]


# Récupération des fichiers générés à partir des API YouTube
folder_candidats = './data_sources/stats_videos/2022_03_26/'
# Création d'un datafram qui agrège les fichiers
dt_candidats = mod_810fr.mb_ds()
# Génération d'un dataframe de vidéos et d'un dataframe de chaîne (exemple dans les blocs qui suivent)
videos_candidats_toutes, chaines_candidats_toutes  = dt_candidats.get_full_data_candidat(folder = folder_candidats)
# Generation d'un dataframe par mois sous la forme d'une liste de [{"mois" : moisassocie, "dataframe" : dataframedumois}]
liste_dtf_mensuel = amd810.un_dtf_par_mois(dtfglobal=videos_candidats_toutes, list_dict_dates=liste_mois)

Imported 11 channel data


In [2]:
candidats = chaines_candidats_toutes.candidat.unique()
candidats

array(['Eric Zemmour', 'Marine Le Pen', 'Anne Hidalgo',
       'Valérie Pécresse', 'Jean Lassalle', 'Jean-Luc Mélenchon',
       'Nathalie Arthaud', 'Nicolas Dupont-Aignan', 'Fabien Roussel',
       'Avec vous', 'Yannick Jadot'], dtype=object)

In [3]:
# videos_candidats_toutes.head(20)

- 1 top tags global, avec la part pour chaque mois
- 1 top tags par mois


- Necessite de parler en nombre de videos Taggees
Qui tags et pourquoi.

In [4]:
datemin = liste_mois[0]["dates"]["datemin"]
datemax = liste_mois[-1]["dates"]["datemax"]

# Jean Lassalle

In [5]:
candidat = "Jean-Luc Mélenchon"
lst_exceptions = []

In [6]:
# Récupère seulement les vidéos d'un candidat dans le dtf globale sur la base du nom du candidat et des dates limites choisies
videos_can = amd810.get_candidat_subdtf(dtfglobal=videos_candidats_toutes, candidat=candidat, datemin=datemin, datemax=datemax)
# Crée une liste de dataframe par mois
dtf_mensuel_candidat = amd810.un_dtf_par_mois(dtfglobal=videos_can, list_dict_dates=liste_mois)
# Donne les tags les plus utilisés sur la période de temps écoulée entre datemin et datemax
top_tags_global = tv810.liste_top_tags_de_chaines (dtfv = videos_can, top = 15, liste_exceptions= lst_exceptions,graph='non')

In [7]:
dtf_tauxtags_mois = tv810.taux_tags_par_mois(listedtfmois=dtf_mensuel_candidat)
dtf_tauxtags_mois.head(20)

,Janvier,Février,Mars
avec tags,96.4286,88.8889,100
sans tags,3.57143,11.1111,0


In [8]:
videos_can = videos_candidats_toutes[videos_candidats_toutes['candidat'] == candidat]
videos_can = videos_can[videos_can['dateVideo'] > datetime.datetime(2022,1,1,0,0,0)]
videos_can = videos_can[videos_can['dateVideo'] < datetime.datetime(2022,4,1,0,0,0)]
print("total videos : " + str(videos_can.shape[0]))

globaltagdtf = tv810.turnserie_intodtf(nomcolonne="Total", serie_data=top_tags_global)

total videos : 78


In [9]:
for dtf in dtf_mensuel_candidat :
    mois_courant  = dtf['mois']
    dtfmois  = dtf['dataframe']
    print(mois_courant + " : " + str(dtfmois.shape[0]))

Janvier : 28
Février : 27
Mars : 23


In [10]:
# print(top_tags_global)

In [11]:
dtf = tv810.nb_tags_par_mois(listedtfmois=dtf_mensuel_candidat,toptagsglobal=top_tags_global, lst_except=lst_exceptions)
dtf = dtf.join(globaltagdtf)
dtf = dtf.sort_values(by = ["Total"], ascending = False)
dtf.head(20)

,Janvier,Février,Mars,Total
mélenchon,14,10,21,45
ukraine,0,13,16,29
russie,0,10,14,25
macron,5,9,8,22
poutine,0,8,14,22
guerre,0,5,8,13
smic,0,6,7,13
otan,2,8,2,12
prix,5,2,4,11
europe,4,4,2,10


In [12]:
tfb = tv810.part_tags_par_mois(listedtfmois=dtf_mensuel_candidat,toptagsglobal=top_tags_global, lst_except=lst_exceptions)

In [13]:
delta_taux = tv810.gen_delta(dtfsource=tfb, colfin="Mars", coldeb="Janvier", nomcol="Delta")
delta_taux.head(20)

,Janvier,Février,Mars,Delta
nucléaire,0,23.0769,23.8095,23.809524
essence,14.2857,15.3846,23.8095,9.523810
école,21.4286,15.3846,19.0476,-2.380952
retraite,14.2857,15.3846,23.8095,9.523810
hôpital,14.2857,30.7692,14.2857,0.000000
europe,28.5714,30.7692,9.52381,-19.047619
prix,35.7143,15.3846,19.0476,-16.666667
otan,14.2857,61.5385,9.52381,-4.761905
guerre,0,38.4615,38.0952,38.095238
smic,0,46.1538,33.3333,33.333333
